# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397837953552                   -0.90    5.2   28.0ms
  2   -8.400239221851       -2.62       -1.74    1.0   19.6ms
  3   -8.400406762017       -3.78       -2.99    1.5   20.8ms
  4   -8.400427865154       -4.68       -3.00    3.2   78.8ms
  5   -8.400427986390       -6.92       -3.10    1.0   20.0ms
  6   -8.400428146866       -6.79       -4.66    1.0   19.7ms
  7   -8.400428151866       -8.30       -4.51    2.5   23.9ms
  8   -8.400428152192       -9.49       -5.40    1.0   20.4ms
  9   -8.400428152208      -10.80       -6.23    1.8   22.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397854873846                   -0.90           5.2    1.78s
  2   -8.400387017851       -2.60       -1.78   0.80    2.2    536ms
  3   -8.400423564665       -4.44       -2.99   0.80    1.0    229ms
  4   -8.400428105894       -5.34       -3.47   0.80    2.5   21.1ms
  5   -8.400428147837       -7.38       -4.92   0.80    1.0    109ms
  6   -8.400428152202       -8.36       -5.44   0.80    3.2   24.2ms
  7   -8.400428152209      -11.16       -6.69   0.80    1.5   19.2ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.836112663745                   -1.06    3.42s
  2   -1.882345719382        0.43       -0.64    147ms
  3   -4.522710342541        0.42       -0.36    106ms
  4   -6.068290572238        0.19       -0.48   44.6ms
  5   -7.452440078417        0.14       -0.67   45.0ms
  6   -7.723838610857       -0.57       -1.42   33.4ms
  7   -7.970254119942       -0.61       -1.54   33.1ms
  8   -8.087629294013       -0.93       -1.60   65.1ms
  9   -8.178094409800       -1.04       -2.09   33.3ms
 10   -8.238744356157       -1.22       -1.92   33

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397834186370                   -0.90    5.0   27.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427981352                   -1.79    8.27s
  2   -8.400428152209       -6.77       -4.03    3.17s
  3   -8.400428152209   +    -Inf       -7.83    124ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.61829710086624e-7
|ρ_newton - ρ_scfv| = 3.6888884762685887e-7
|ρ_newton - ρ_dm|   = 3.1025617090497247e-6
